Using Interactive Brokers API to get data. Only focusing on data pulling, as there are multiple avenues that can be taken for different types of data being pulled (stock, options, historical, live, etc.). All data processing located elsewhere

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, time, datetime


import warnings
from IPython.core.display import display, HTML
warnings.filterwarnings("ignore") # suppress warnings because theyre annoying and i like living on the edge
pd.set_option('display.max_columns',100) # so i can see all columns
np.set_printoptions(linewidth = 220) # increase numpy array print
display(HTML("<style>.container { width:80% !important; }</style>")) # widen jupyter notebook blocks

In [ ]:
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract as IBcontract
from ibapi.ticktype import TickTypeEnum
import time 

def rsi(prices, n = 14, avg_method = 'simple'):
    # given a list of sequential price changes (prices), use len(prices) changes to calculate rsi
    # prices = numpy array
    # n = int
    # avg_method = str
    
    assert len(prices) > n, 'list of price delta less than rsi period'
    
    rsi_list = [np.nan] * n
    
    n_prices = len(prices)
    if avg_method == 'simple':
        alpha = 1
    elif avg_method == 'exponential':
        alpha = 2 / (n+1)
    elif avg_method == 'wilder':
        alpha = 1 / n
    for i in range(n, n_prices):
        vals = prices[(i-n):i]
        U = np.sum(vals * (vals > 0).astype(int)) / n_prices
        D = -1 * np.sum(vals * (vals < 0).astype(int)) / n_prices
        if i == n:
            avgU = U
            avgD = D
        else:
            U = np.sum(vals * (vals > 0).astype(int)) / n_prices
            D = -1 * np.sum(vals * (vals < 0).astype(int)) / n_prices
            avgU = alpha * U + (1 - alpha) * prevU
            avgD = alpha * D + (1 - alpha) * prevD
        prevU = avgU
        prevD = avgD
        rs = avgU / avgD
        rsi = 100.0 - 100.0 / (1 + rs)
        rsi_list.append(rsi)
    return rsi_list

class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self,self)
        self.ticker_dict = {}

    def error(self, reqId, errorCode, errorString):
        print("Error: ", reqId, " ", errorCode, " ", errorString)
    
#     def tickPrice(self, reqId, tickType, price, attrib):
# #         print("Tick Price. Ticker Id:", reqId, "tickType:", TickTypeEnum.to_str(tickType), "Price:", price, end=' \n')
# #         print('time: ' + str(datetime.now()) )
# #         time.sleep(5)
#         pass
#     def tickSize(self, reqId, tickType, size):
#         print("Tick Size. Ticker Id:", reqId, "tickType:", TickTypeEnum.to_str(tickType), "Size:", size,'\n')
        
    ## realtimebar stuff
    def realtimeBar(self, reqId, time:int, open_: float, high: float, low: float, close: float,
                        volume: int, wap: float, count: int):
        print(self.ticker_dict[reqId],time, open_, high, low, close, volume, wap, count)

    def start_reqRealTimeBars(self, reqId, contract, barSize, whatToShow,useRTH,realTimeBarsOptions):
        self.ticker_dict[reqId] = contract.symbol
        print('Starting requests for ' + str(self.ticker_dict[reqId]))
        self.reqRealTimeBars(reqId, contract, barSize, whatToShow,useRTH,realTimeBarsOptions)


def basicContract(symbol):
    contract = IBcontract()
    contract.symbol = symbol
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    contract.primaryExchange = "NASDAQ"
    return contract

app = TestApp()
app.connect("127.0.0.1", 7497, 1)
contract = basicContract('AAPL')
app.start_reqRealTimeBars(1,contract,5,'TRADES',0,[])
# contract = basicContract('MSFT')
# app.start_reqRealTimeBars(2,contract,5,'MIDPOINT',0,[])

# app.reqMarketDataType(1)
# app.reqMktData(1,contract, "233", False, False, [])
# app.reqRealTimeBars(1,contract,5,'MIDPOINT',0,[])


app.run()

Starting requests for AAPL
Error:  -1   2104   Market data farm connection is OK:usfarm.nj
Error:  -1   2104   Market data farm connection is OK:cashfarm
Error:  -1   2104   Market data farm connection is OK:usfarm
Error:  -1   2106   HMDS data farm connection is OK:ushmds
Error:  -1   2158   Sec-def data farm connection is OK:secdefnj
AAPL 1604698415 118.64 118.64 118.64 118.64 1 118.64 1
AAPL 1604698420 118.64 118.64 118.64 118.64 0 118.64 0
AAPL 1604698425 118.66 118.66 118.66 118.66 3 118.66 2
AAPL 1604698430 118.66 118.66 118.66 118.66 0 118.66 0
AAPL 1604698435 118.66 118.66 118.66 118.66 0 118.66 0
AAPL 1604698440 118.66 118.66 118.66 118.66 0 118.66 0
AAPL 1604698445 118.66 118.66 118.66 118.66 0 118.66 0
AAPL 1604698450 118.66 118.66 118.66 118.66 0 118.66 0


In [2]:

from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract as IBcontract
from threading import Thread
import queue
import datetime

DEFAULT_HISTORIC_DATA_ID=50
DEFAULT_MARKET_DATA_ID=47
DEFAULT_GET_CONTRACT_ID=43

## marker for when queue is finished
FINISHED = object()
STARTED = object()
TIME_OUT = object()

class finishableQueue(object):

    def __init__(self, queue_to_finish):

        self._queue = queue_to_finish
        self.status = STARTED

    def get(self, timeout):
        """
        Returns a list of queue elements once timeout is finished, or a FINISHED flag is received in the queue
        :param timeout: how long to wait before giving up
        :return: list of queue elements
        """
        contents_of_queue=[]
        finished=False

        while not finished:
            try:
                current_element = self._queue.get(timeout=timeout)
                if current_element is FINISHED:
                    finished = True
                    self.status = FINISHED
                else:
                    contents_of_queue.append(current_element)
                    ## keep going and try and get more data

            except queue.Empty:
                ## If we hit a time out it's most probable we're not getting a finished element any time soon
                ## give up and return what we have
                finished = True
                self.status = TIME_OUT


        return contents_of_queue

    def timed_out(self):
        return self.status is TIME_OUT


def _nan_or_int(x):
    if not np.isnan(x):
        return int(x)
    else:
        return x

class stream_of_ticks(list):
    """
    Stream of ticks
    """

    def __init__(self, list_of_ticks):
        super().__init__(list_of_ticks)

    def as_pdDataFrame(self):

        if len(self)==0:
            ## no data; do a blank tick
            return tick(datetime.datetime.now()).as_pandas_row()

        pd_row_list=[tick.as_pandas_row() for tick in self]
        pd_data_frame=pd.concat(pd_row_list)

        return pd_data_frame


class tick(object):
    """
    Convenience method for storing ticks
    Not IB specific, use as abstract
    """
    def __init__(self, timestamp, bid_size=np.nan, bid_price=np.nan,
                 ask_size=np.nan, ask_price=np.nan,
                 last_trade_size=np.nan, last_trade_price=np.nan,
                 ignorable_tick_id=None):

        ## ignorable_tick_id keyword must match what is used in the IBtick class

        self.timestamp=timestamp
        self.bid_size=_nan_or_int(bid_size)
        self.bid_price=bid_price
        self.ask_size=_nan_or_int(ask_size)
        self.ask_price=ask_price
        self.last_trade_size=_nan_or_int(last_trade_size)
        self.last_trade_price=last_trade_price

    def __repr__(self):
        return self.as_pandas_row().__repr__()

    def as_pandas_row(self):
        """
        Tick as a pandas dataframe, single row, so we can concat together
        :return: pd.DataFrame
        """

        attributes=['bid_size','bid_price', 'ask_size', 'ask_price',
                    'last_trade_size', 'last_trade_price']

        self_as_dict=dict([(attr_name, getattr(self, attr_name)) for attr_name in attributes])

        return pd.DataFrame(self_as_dict, index=[self.timestamp])


class IBtick(tick):
    """
    Resolve IB tick categories
    """

    def __init__(self, timestamp, tickid, value):

        resolve_tickid=self.resolve_tickids(tickid)
        super().__init__(timestamp, **dict([(resolve_tickid, value)]))

    def resolve_tickids(self, tickid):

        tickid_dict=dict([("0", "bid_size"), ("1", "bid_price"), ("2", "ask_price"), ("3", "ask_size"),
                          ("4", "last_trade_price"), ("5", "last_trade_size")])

        if str(tickid) in tickid_dict.keys():
            return tickid_dict[str(tickid)]
        else:
            # This must be the same as the argument name in the parent class
            return "ignorable_tick_id"






class TestWrapper(EWrapper):
    """
    The wrapper deals with the action coming back from the IB gateway or TWS instance
    We override methods in EWrapper that will get called when this action happens, like currentTime
    Extra methods are added as we need to store the results in this object
    """

    def __init__(self):
        self._my_contract_details = {}
        self._my_historic_data_dict = {}
        self._my_market_data_dict = {}
        
    ## error handling code
    def init_error(self):
        error_queue=queue.Queue()
        self._my_errors = error_queue

    def get_error(self, timeout=5):
        if self.is_error():
            try:
                return self._my_errors.get(timeout=timeout)
            except queue.Empty:
                return None

        return None

    def is_error(self):
        an_error_if=not self._my_errors.empty()
        return an_error_if

    def error(self, id, errorCode, errorString):
        ## Overriden method
        errormsg = "IB error id %d errorcode %d string %s" % (id, errorCode, errorString)
        self._my_errors.put(errormsg)


    ## get contract details code
    def init_contractdetails(self, reqId):
        contract_details_queue = self._my_contract_details[reqId] = queue.Queue()

        return contract_details_queue

    def contractDetails(self, reqId, contractDetails):
        ## overridden method

        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(contractDetails)

    def contractDetailsEnd(self, reqId):
        ## overriden method
        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(FINISHED)

    ## Historic data code
    def init_historicprices(self, tickerid):
        historic_data_queue = self._my_historic_data_dict[tickerid] = queue.Queue()

        return historic_data_queue


    def historicalData(self, tickerid , bar):

        ## Overriden method
        ## Note I'm choosing to ignore barCount, WAP and hasGaps but you could use them if you like
        bardata=(bar.date, bar.open, bar.high, bar.low, bar.close, bar.volume)

        historic_data_dict=self._my_historic_data_dict

        ## Add on to the current data
        if tickerid not in historic_data_dict.keys():
            self.init_historicprices(tickerid)

        historic_data_dict[tickerid].put(bardata)

    def historicalDataEnd(self, tickerid, start:str, end:str):
        ## overriden method

        if tickerid not in self._my_historic_data_dict.keys():
            self.init_historicprices(tickerid)

        self._my_historic_data_dict[tickerid].put(FINISHED)

    ## market data

    def init_market_data(self, tickerid):
        market_data_queue = self._my_market_data_dict[tickerid] = queue.Queue()

        return market_data_queue


    def get_time_stamp(self):
        ## Time stamp to apply to market data
        ## We could also use IB server time
        return datetime.datetime.now()


    def tickPrice(self, tickerid , tickType, price, attrib):
        ##overriden method

        ## For simplicity I'm ignoring these but they could be useful to you...
        ## See the documentation http://interactivebrokers.github.io/tws-api/md_receive.html#gsc.tab=0
        # attrib.canAutoExecute
        # attrib.pastLimit

        this_tick_data=IBtick(self.get_time_stamp(),tickType, price)
        self._my_market_data_dict[tickerid].put(this_tick_data)


    def tickSize(self, tickerid, tickType, size):
        ## overriden method

        this_tick_data=IBtick(self.get_time_stamp(), tickType, size)
        self._my_market_data_dict[tickerid].put(this_tick_data)


    def tickString(self, tickerid, tickType, value):
        ## overriden method

        ## value is a string, make it a float, and then in the parent class will be resolved to int if size
        this_tick_data=IBtick(self.get_time_stamp(),tickType, float(value))
        self._my_market_data_dict[tickerid].put(this_tick_data)


    def tickGeneric(self, tickerid, tickType, value):
        ## overriden method

        this_tick_data=IBtick(self.get_time_stamp(),tickType, value)
        self._my_market_data_dict[tickerid].put(this_tick_data)

class TestClient(EClient):
    """
    The client method
    We don't override native methods, but instead call them from our own wrappers
    """
    def __init__(self, wrapper):
        ## Set up with a wrapper inside
        EClient.__init__(self, wrapper)
        self._market_data_q_dict = {}


    def resolve_ib_contract(self, ibcontract, reqId=DEFAULT_GET_CONTRACT_ID):

        """
        From a partially formed contract, returns a fully fledged version
        :returns fully resolved IB contract
        """

        ## Make a place to store the data we're going to return
        contract_details_queue = finishableQueue(self.init_contractdetails(reqId))

        print("Getting full contract details from the server... ")

        self.reqContractDetails(reqId, ibcontract)

        ## Run until we get a valid contract(s) or get bored waiting
        MAX_WAIT_SECONDS = 30
        new_contract_details = contract_details_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if contract_details_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")

        if len(new_contract_details)==0:
            print("Failed to get additional contract details: returning unresolved contract")
            return ibcontract

        if len(new_contract_details)>1:
            print("got multiple contracts using first one")

        new_contract_details=new_contract_details[0]

        resolved_ibcontract=new_contract_details.contract

        return resolved_ibcontract


    def get_IB_historical_data(self, ibcontract, durationStr="1 D", barSizeSetting="5 secs",
                               tickerid=DEFAULT_HISTORIC_DATA_ID):

        """
        Returns historical prices for a contract, up to today
        ibcontract is a Contract
        :returns list of prices in 4 tuples: Open high low close volume
        """


        ## Make a place to store the data we're going to return
        historic_data_queue = finishableQueue(self.init_historicprices(tickerid))

        # Request some historical data. Native method in EClient
        self.reqHistoricalData(
            tickerid,  # tickerId,
            ibcontract,  # contract,
            datetime.datetime.today().strftime("%Y%m%d %H:%M:%S %Z"),  # endDateTime,
            durationStr,  # durationStr,
            barSizeSetting,  # barSizeSetting,
            "TRADES",  # whatToShow,
            1,  # useRTH,
            1,  # formatDate
            False,  # KeepUpToDate <<==== added for api 9.73.2
            [] ## chartoptions not used
        )


        ## Wait until we get a completed data, an error, or get bored waiting
        MAX_WAIT_SECONDS = 200
        print("Getting historical data from the server... could take %d seconds to complete " % MAX_WAIT_SECONDS)

        historic_data = historic_data_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if historic_data_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")
        self.cancelHistoricalData(tickerid)


        return historic_data
    



    def start_getting_IB_market_data(self, resolved_ibcontract, tickerid=DEFAULT_MARKET_DATA_ID):
        """
        Kick off market data streaming
        :param resolved_ibcontract: a Contract object
        :param tickerid: the identifier for the request
        :return: tickerid
        """

        self._market_data_q_dict[tickerid] = self.wrapper.init_market_data(tickerid)
        self.reqMktData(tickerid, resolved_ibcontract, "", False, False, [])

        return tickerid

    def stop_getting_IB_market_data(self, tickerid):
        """
        Stops the stream of market data and returns all the data we've had since we last asked for it
        :param tickerid: identifier for the request
        :return: market data
        """

        ## native EClient method
        self.cancelMktData(tickerid)

        ## Sometimes a lag whilst this happens, this prevents 'orphan' ticks appearing
        time.sleep(5)

        market_data = self.get_IB_market_data(tickerid)

        ## output ay errors
        while self.wrapper.is_error():
            print(self.get_error())

        return market_data

    def get_IB_market_data(self, tickerid):
        """
        Takes all the market data we have received so far out of the stack, and clear the stack
        :param tickerid: identifier for the request
        :return: market data
        """

        ## how long to wait for next item
        MAX_WAIT_MARKETDATEITEM = 5
        market_data_q = self._market_data_q_dict[tickerid]

        market_data=[]
        finished=False

        while not finished:
            try:
                market_data.append(market_data_q.get(timeout=MAX_WAIT_MARKETDATEITEM))
            except queue.Empty:
                ## no more data
                finished=True

        return stream_of_ticks(market_data)

class TestApp(TestWrapper, TestClient):
    def __init__(self, ipaddress, portid, clientid):
        error_queue=queue.Queue()
        self._my_errors = error_queue
        
        TestWrapper.__init__(self)
        TestClient.__init__(self, wrapper=self)

        self.connect(ipaddress, portid, clientid)

        thread = Thread(target = self.run)
        thread.start()

        setattr(self, "_thread", thread)

        self.init_error()


#if __name__ == '__main__':

# app = TestApp("127.0.0.1", 7497, 1)

# ibcontract = IBcontract()
# ibcontract.secType = "STK"
# ibcontract.lastTradeDateOrContractMonth="202010"
# ibcontract.symbol="AAPL"
# ibcontract.exchange="SMART"

# resolved_ibcontract=app.resolve_ib_contract(ibcontract)

# historic_data = app.get_IB_historical_data(resolved_ibcontract)

# print(historic_data)

# app.disconnect()

In [3]:
app = TestApp("127.0.0.1", 7497, 1)
ibcontract = IBcontract()
ibcontract.secType = "STK"
# ibcontract.lastTradeDateOrContractMonth="202011"
ibcontract.symbol="AAPL"
ibcontract.exchange="SMART"
ibcontract.currency = "USD"
ibcontract.primaryExchange = "NASDAQ"
resolved_ibcontract=app.resolve_ib_contract(ibcontract)
# print(resolved_ibcontract)
# historic_data = app.get_IB_historical_data(resolved_ibcontract, durationStr = "1 D", barSizeSetting = "1 min")

# real_data = app.get_rt_bar(resolved_ibcontract)
# print(real_data)
# print(historic_data)

# app.disconnect()

Getting full contract details from the server... 
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfarm.nj
IB error id -1 errorcode 2104 string Market data farm connection is OK:cashfarm
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:euhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:fundfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds
IB error id -1 errorcode 2158 string Sec-def data farm connection is OK:secdefnj


In [10]:
import time
# app = TestApp("127.0.0.1", 7497, 1)
market_data2 == []

while market_data2 == []:
    tickerid = app.start_getting_IB_market_data(resolved_ibcontract)
    # time.sleep(5)
    # market_data1 = app.get_IB_market_data(tickerid)
    time.sleep(5)
    market_data2 = app.stop_getting_IB_market_data(tickerid)
    # app.disconnect()
print(market_data2)

IB error id 47 errorcode 504 string Not connected
IB error id 47 errorcode 504 string Not connected
IB error id 47 errorcode 504 string Not connected
IB error id 47 errorcode 504 string Not connected
IB error id 47 errorcode 504 string Not connected
IB error id 47 errorcode 504 string Not connected
IB error id 47 errorcode 504 string Not connected
IB error id 47 errorcode 504 string Not connected
IB error id 47 errorcode 504 string Not connected
IB error id 47 errorcode 504 string Not connected


KeyboardInterrupt: 

In [9]:
market_data2

[]

In [ ]:
EClient.reqMktData.__code__.co_varnames

In [ ]:
market_data2

In [ ]:
import pickle

# save csv for tableau or otherwise
work_dir = 'D:/general_data/stock'
with open(work_dir + 'aapl_20201002.pkl','wb') as pickle_out:
    pickle.dump(historic_data, pickle_out)
# df.to_csv(work_dir + '/aapl_20200928.csv', index = False)